In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, models, layers
import pandas as pd

In [4]:
df = pd.read_csv("/content/heart.csv")

In [69]:
df

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,63.0,1.0,3.0,145.0,233.0,1.0,0.0,150.0,0.0,2.3,0.0,0.0,1.0,1.0
1,37.0,1.0,2.0,130.0,250.0,0.0,1.0,187.0,0.0,3.5,0.0,0.0,2.0,1.0
2,41.0,0.0,1.0,130.0,204.0,0.0,0.0,172.0,0.0,1.4,2.0,0.0,2.0,1.0
3,56.0,1.0,1.0,120.0,236.0,0.0,1.0,178.0,0.0,0.8,2.0,0.0,2.0,1.0
4,57.0,0.0,0.0,120.0,354.0,0.0,1.0,163.0,1.0,0.6,2.0,0.0,2.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,57.0,0.0,0.0,140.0,241.0,0.0,1.0,123.0,1.0,0.2,1.0,0.0,3.0,0.0
299,45.0,1.0,3.0,110.0,264.0,0.0,1.0,132.0,0.0,1.2,1.0,0.0,3.0,0.0
300,68.0,1.0,0.0,144.0,193.0,1.0,1.0,141.0,0.0,3.4,1.0,2.0,3.0,0.0
301,57.0,1.0,0.0,130.0,131.0,0.0,1.0,115.0,1.0,1.2,1.0,1.0,3.0,0.0


In [10]:
df.duplicated().sum()

0

In [9]:
df.drop_duplicates(inplace=True)

In [12]:
df.isnull().sum()

age         0
sex         0
cp          0
trtbps      0
chol        0
fbs         0
restecg     0
thalachh    0
exng        0
oldpeak     0
slp         0
caa         0
thall       0
output      0
dtype: int64

In [25]:
len(y_train), len(x_test)

(241, 61)

In [16]:
from sklearn.model_selection import train_test_split

In [18]:
x = df.drop("output", axis = 1)
y = df["output"]

In [31]:
df = df.astype("float32")

In [32]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [44]:
class FullConnLayer(layers.Layer):
  def __init__(self, out_channels):
    super(FullConnLayer, self).__init__()
    self.Dense = layers.Dense(out_channels)
    self.bn = layers.BatchNormalization()

  def call(self, cells, training = False):
    x = self.Dense(cells)
    x = self.bn(x, training = training)
    x = tf.nn.relu(x)
    return x

In [63]:
model = keras.Sequential(
    [FullConnLayer(16), FullConnLayer(32), FullConnLayer(64), layers.Flatten(), layers.Dense(2),]
)

In [64]:
model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True),
    optimizer = keras.optimizers.Adam(learning_rate = 0.001),
    metrics = ["accuracy"]
)

In [67]:
model.fit(x_train, y_train, epochs = 50, batch_size=32)

Epoch 1/50
8/8 [==============================] - 0s 8ms/step - loss: 0.5336 - accuracy: 0.7303
Epoch 2/50
8/8 [==============================] - 0s 7ms/step - loss: 0.5159 - accuracy: 0.7552
Epoch 3/50
8/8 [==============================] - 0s 14ms/step - loss: 0.5387 - accuracy: 0.7261
Epoch 4/50
8/8 [==============================] - 0s 9ms/step - loss: 0.4899 - accuracy: 0.7718
Epoch 5/50
8/8 [==============================] - 0s 14ms/step - loss: 0.5267 - accuracy: 0.7469
Epoch 6/50
8/8 [==============================] - 0s 13ms/step - loss: 0.4886 - accuracy: 0.7884
Epoch 7/50
8/8 [==============================] - 0s 11ms/step - loss: 0.4933 - accuracy: 0.7801
Epoch 8/50
8/8 [==============================] - 0s 10ms/step - loss: 0.4841 - accuracy: 0.7676
Epoch 9/50
8/8 [==============================] - 0s 9ms/step - loss: 0.5223 - accuracy: 0.7842
Epoch 10/50
8/8 [==============================] - 0s 9ms/step - loss: 0.4982 - accuracy: 0.7510
Epoch 11/50
8/8 [=================

In [68]:
model.evaluate(x_test, y_test)

2/2 [==============================] - 0s 7ms/step - loss: 0.5767 - accuracy: 0.7377


[0.5766789317131042, 0.7377049326896667]